In [1]:
import os
import itertools
from glob import glob
import multiprocessing as mp
import random
import numpy as np
import pandas as pd
import torch
import torch.optim as optim
from torch import nn
from torch.nn import init
from torch.optim import Adam, lr_scheduler
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm
from torch_geometric.data import Data, Batch
from torch_geometric.data import DataLoader
from torch_geometric.nn import DataParallel
from torch_geometric.data import DataListLoader
from dataset import PosteraDataset
from sgcnn_model import PotentialNetParallel, GraphThreshold
from sklearn.preprocessing import StandardScaler
from torch.utils.data import ConcatDataset, SubsetRandomSampler

/g/g20/wilfong2/.conda/envs/my-rdkit-env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# parameters 
dataset_name = 'Postera'
batch_size = 8
covalent_gather_width = 16
non_covalent_gather_width = 12
covalent_k =2
non_covalent_k = 2
#epochs=100 
covalent_threshold = 1.5
non_covalent_threshold = 4.5
lr = 0.001
checkpoint = True
checkpoint_dir = 'g/g20/wilfong2/DSSI/sgcnn/new'
num_workers=8
train_data = 'postera_protease2_pos_neg_train.hdf5'
val_data = 'postera_protease2_pos_neg_val.hdf5'
use_docking = False
feature_size=19

In [3]:
def worker_init_fn(worker_id):
    np.random.seed(int(0))

def collate_fn_none_filter(batch):
    return [x for x in batch if x is not None]

In [4]:
model = (PotentialNetParallel(
    in_channels=feature_size,
    out_channels=1,
    non_covalent_gather_width=non_covalent_gather_width,
    covalent_gather_width=covalent_gather_width,
    covalent_k=covalent_k,
    non_covalent_k=non_covalent_k,
    covalent_neighbor_threshold=covalent_threshold,
    non_covalent_neighbor_threshold=non_covalent_threshold, device_ids=None)).float()

In [5]:
losses = []
def train(epoch):
    print('\nEpoch : %d'%epoch)
    """
    input args
    
    loss: loss function 
    dataloader: 
    device: cuda or cpu 
    """
    bs = batch_size # set up batch size for training & val
    #loss = []
    train_dataset = []
    path = os.path.join("postera_protease2_pos_neg_train.hdf5")
    train_dataset.append(PosteraDataset(path, features=False))
    #for data in train_data:
        #train_dataset.append(PosteraDataset(data)) # using dataset.py to prepare data
    #for data in args.val_data:
        #val_dataset.append(PosteraDataset(data)) # dataset.py to prepare validation data
    train_dataloader = DataListLoader(train_dataset,batch_size=bs, shuffle=False, worker_init_fn=worker_init_fn) # laoding data
    # can add drop_last=True in the instance of shuffling dataset
     #val_dataloader = DataListLoader(val_dataset, batch_size=bs, shuffle=False, worker_init_fn=worker_init_fn) # load val data
    model = (PotentialNetParallel(
        in_channels=feature_size,
        out_channels=1,
        non_covalent_gather_width=non_covalent_gather_width,
        covalent_k=covalent_k,
        non_covalent_k=non_covalent_k,
        covalent_neighbor_threshold=covalent_threshold,
        non_covalent_neighbor_threshold=non_covalent_threshold, device_ids=None)).float()
    
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    model.to(device)
    model.train()
    
    loss_fn = nn.CrossEntropyLoss()
    optimizer=optim.SGD(model.parameters(), lr=0.001,momentum=0.9)
    #loss_fn = nn.CrossEntropyLoss() # loss fucntion
    #optimizer=optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # might need to add momentum?
    
    #print('\nEpoch : d%'%epoch)
    
    running_loss=0
    correct=0
    total=0
    
    for batch in tqdm(train_dataloader):
        batch = [x for x in batch if x is not None]
        optimizer.zero_grad()
        
        data = [x[2] for x in batch]
        y_ = model(data)
        y = torch.cat([x[2].y for x in batch])
        
        loss = criterion(y.float(), y_cpu().float())
        losses.append(loss.cpu().data.item())
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
      
        train_loss=running_loss/len(trainloader)
        accu=100.*correct/total

        train_accu.append(accu)
        train_losses.append(train_loss)
        print('Train Loss: %.3f | Accuracy: %.3f'%(train_loss,accu))

    # calculate loss


In [6]:
epochs = 10
for epoch in range(1,epochs+1):
    train(epoch)

/g/g20/wilfong2/.conda/envs/my-rdkit-env/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataListLoader' is deprecated, use 'loader.DataListLoader' instead
  warnings.warn(out)



Epoch : 1


  0%|          | 0/1 [00:00<?, ?it/s]


AttributeError: 'list' object has no attribute 'edge_index'